In [1]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # or "tensorflow" or "torch"

import keras_hub
import keras
import tensorflow as tf
import time

keras.mixed_precision.set_global_policy("mixed_float16")

2024-11-08 10:54:35.365002: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731043475.379700   10918 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731043475.383826   10918 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/home/aarav/anaconda3/envs/keras-jax/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json
import tensorflow as tf

In [3]:
import pandas as pd

In [4]:
from datasets import load_dataset
dataset = load_dataset('reddit_tifu','short')

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ups', 'num_comments', 'upvote_ratio', 'score', 'documents', 'tldr', 'title'],
        num_rows: 79740
    })
})

In [6]:
import numpy as np

In [7]:
data = np.array(dataset['train'][0:]['documents'])

In [8]:
data

array(["i was on skype on my tablet as i went to the toilet iming a friend. i don't multitask very well, so i forgot one of the most important things to do before pooping. i think the best part was when i realised and told my mate who just freaked out because i was talking to him on the john!",
       'this actually happened a couple of years ago. i grew up in germany where i went to a german secondary school that went from 5th to 13th grade (we still had 13 grades then, they have since changed that). my school was named after anne frank and we had a club that i was very active in from 9th grade on, which was dedicated to teaching incoming 5th graders about anne franks life, discrimination, anti-semitism, hitler, the third reich and that whole spiel. basically a day where the students\' classes are cancelled and instead we give them an interactive history and social studies class with lots of activities and games. \n\nthis was my last year at school and i already had a lot of experienc

In [9]:
data.shape

(79740,)

In [10]:
dataset = tf.data.Dataset.from_tensor_slices(data)

2024-11-08 10:54:53.697405: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1731043493.697560   10918 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2278 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [11]:
train_ds = dataset.batch(4).cache().prefetch(tf.data.AUTOTUNE)

In [12]:
train_ds.take(1)

<_TakeDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>

In [13]:
for string in train_ds.take(1):
    print(string)

tf.Tensor(
[b"i was on skype on my tablet as i went to the toilet iming a friend. i don't multitask very well, so i forgot one of the most important things to do before pooping. i think the best part was when i realised and told my mate who just freaked out because i was talking to him on the john!"
 b'this actually happened a couple of years ago. i grew up in germany where i went to a german secondary school that went from 5th to 13th grade (we still had 13 grades then, they have since changed that). my school was named after anne frank and we had a club that i was very active in from 9th grade on, which was dedicated to teaching incoming 5th graders about anne franks life, discrimination, anti-semitism, hitler, the third reich and that whole spiel. basically a day where the students\' classes are cancelled and instead we give them an interactive history and social studies class with lots of activities and games. \n\nthis was my last year at school and i already had a lot of experienc

2024-11-08 10:54:56.768090: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [14]:
preprocessor = keras_hub.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = keras_hub.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

In [15]:
train_ds = train_ds.take(500)
num_epochs = 1
import keras
# Linearly decaying learning rate.
learning_rate = keras.optimizers.schedules.PolynomialDecay(
    5e-5,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

gpt2_lm.fit(train_ds, epochs=num_epochs)

2024-11-08 10:55:03.852898: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT64 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
2024-11-08 10:55:03.993559: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-11-08 10:55:25.125917: W external/xla/xla/hlo/transforms/simplifiers/hlo_rematerialization.cc:3020] Can't reduce memory use below 1.39GiB (1491324481 bytes) by rematerialization; only reduced to 1.96GiB (2110682477 bytes), down from 2.00GiB (2146223231 bytes) originally


  1/500 ━━━━━━━━━━━━━━━━━━━━ 3:39:33 26s/step - accuracy: 0.2760 - loss: 3.1837

2024-11-08 10:55:39.476165: W external/xla/xla/hlo/transforms/simplifiers/hlo_rematerialization.cc:3020] Can't reduce memory use below 1.39GiB (1491324481 bytes) by rematerialization; only reduced to 1.96GiB (2110682477 bytes), down from 2.00GiB (2146223231 bytes) originally


486/500 ━━━━━━━━━━━━━━━━━━━━ 1s 130ms/step - accuracy: 0.3022 - loss: 3.3286

2024-11-08 10:56:33.948293: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


500/500 ━━━━━━━━━━━━━━━━━━━━ 91s 129ms/step - accuracy: 0.3024 - loss: 3.3262


In [19]:
start = time.time()

output = gpt2_lm.generate("I am machine learning engineer", max_length=200)
print("\nGPT-2 output:")
print(output)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")


GPT-2 output:
I am machine learning engineer and i am a student of machine learning. 

my job is to make machine learning applications for companies and companies with a business model that allows you to make applications for the most common problems in a large number of applications. 

this is where my job title comes in. 

i am a student at a large university and i have been working on a large number of applications.  this is a very small application that i am going to make.  

my main problem is that my application will have a few things that are not in the current version of the software.  i will not be able to make any applications with this software. 

so, this is what the application will look like:

http://www.google.com/appstore/detail?q=http://www.google.com/appstore/detail&q=http://www.google.com/appstore?q=http://www.google
TOTAL TIME ELAPSED: 1.10s
